In [ ]:
from PIL import Image
import numpy as np
import coremltools

# Load the Core ML model
model_path = "bird-resnet34v1.mlpackage"  # Adjust the path to your model
model = coremltools.models.MLModel(model_path)

# Load and prepare the image
img_path = "img/test_imagev3.jpg"
img = Image.open(img_path)
img = img.resize([224, 224], Image.LANCZOS)

# Assume the input name is "input_1" based on your script; adjust if it's different
input_name = model.get_spec().description.input[0].name

# Make a prediction
coreml_out_dict = model.predict({input_name: img})

# Assuming your model outputs include 'classLabel' and a dictionary of probabilities
print("coreml predictions: ")
print("top class label: ", coreml_out_dict["classLabel"])

# Extracting the probabilities dictionary
for out in model.get_spec().description.output:
    if out.type.WhichOneof('Type') == "dictionaryType":
        coreml_dict_name = out.name
        break

coreml_prob_dict = coreml_out_dict[coreml_dict_name]

# Processing the probabilities to find top classes
values_vector = np.array(list(coreml_prob_dict.values()))
keys_vector = list(coreml_prob_dict.keys())
top_3_indices_coreml = np.argsort(-values_vector)[:3]
for i in range(3):
    idx = top_3_indices_coreml[i]
    score_value = values_vector[idx]
    class_id = keys_vector[idx]
    print(f"class name: {class_id}, raw score value: {score_value}")
